Some detailed steps that are not important like combining files into one file etc. are documented in "data_preparation_new.ipynb"

# To Do: 


In [ ]:
# imports
import xarray as xr
import numpy as np 
import pandas as pd  
import matplotlib.pyplot as plt 
import geopandas as gpd # this is used in the hydromt version of aggregation
import glob # to create a dictionary for combininc the NC files
import math
import os
from datetime import datetime as dt

%matplotlib inline

In [ ]:
### new, somehow necessary to import xagg
os.environ["ESMFMKFILE"] = "/Users/jonathanschieren/opt/anaconda3/envs/hydromt/lib/esmf.mk"

In [ ]:
import xagg as xa

Ensemble mean, 0.1 degree regular grid
version 26.0e October 2022, period: 1950-01-01 - 2022-06-30
Link: https://surfobs.climate.copernicus.eu/dataaccess/access_eobs.php#datafiles

TG = daily mean temperature [°C] \
TN = daily minimum temperature [°C] \
TX = daily maximum temperature [°C] \
RR = sum of daily precipitation [mm] \
PP = daily averaged sea level pressure [hPa] \
HU = daily averaged relative humidity [%] \
FG = daily mean wind speed at 10 m height [m/s]\
QQ = surface shortwave downwelling radiation [W/m2] \
ELEV = land surface elevation above sea level [m]

# 1. Load Data & Clip [run only once!]

## 1.1 Load EOBS data

In [ ]:
# load the full files (they are already pre-clipped from the "data_preparation_new.ipynb" notebook)
RR = xr.open_dataarray('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/rr_ens_mean_0.1deg_reg_v26.0e_cl_old.nc', mask_and_scale=False)
TN = xr.open_dataarray('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/tn_ens_mean_0.1deg_reg_v26.0e_cl_old.nc', mask_and_scale=False)
TX = xr.open_dataarray('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/tx_ens_mean_0.1deg_reg_v26.0e_cl_old.nc', mask_and_scale=False)
TG = xr.open_dataarray('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/tg_ens_mean_0.1deg_reg_v26.0e_cl_old.nc', mask_and_scale=False)
PP = xr.open_dataarray('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/pp_ens_mean_0.1deg_reg_v26.0e_cl_old.nc', mask_and_scale=False)
HU = xr.open_dataarray('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/hu_ens_mean_0.1deg_reg_v26.0e_cl_old.nc', mask_and_scale=False)
FG = xr.open_dataarray('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/fg_ens_mean_0.1deg_reg_v26.0e_cl_old.nc', mask_and_scale=False)
QQ = xr.open_dataarray('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/qq_ens_mean_0.1deg_reg_v26.0e_cl_old.nc', mask_and_scale=False)

In [ ]:
# load elevation file
ELEV = xr.open_dataarray('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/elev_ens_0.1deg_reg_v26.0e.nc', mask_and_scale=True)

## 1.2 Load ERA5Land data
Note: 
- Steps like aggregating the annual files are documented in "data_preparation_new.ipynb"

In [ ]:
# load albedo 
ALB = xr.open_dataarray('/Users/jonathanschieren/Desktop/msc_thesis/data/era5land/alb_combi/ERA5_f_alb_mean_total.nc', mask_and_scale=True)

In [ ]:
# sort the latitude values
ALB = ALB.sortby('latitude', ascending=True)

In [ ]:
# load the correct PET which is calculated based on the last hour of each day 
PET_last = xr.open_dataarray('/Users/jonathanschieren/Desktop/msc_thesis/data/era5land/pet_combi/ERA5_pet_total_daily_last.nc', mask_and_scale=True)

In [ ]:
# sort the latitude values
PET_last = PET_last.sortby('latitude', ascending=True)

## 1.1 Reduce spatial extent with rectangle

In [ ]:
# Clip raster data to approximate extent of LamaH dataset
# Corner points in epsg:4236 -> But we did not assign a CRS yet
xmin_C = 8
xmax_C = 18 # would have to be 19 to use basin delineations A and B but we don't need it 
ymin_C = 46
ymax_C = 50

# 
geometries = [
    {
        'type': 'Polygon',
        'coordinates': [[
            [xmin_C, ymin_C],
            [xmin_C, ymax_C],
            [xmax_C, ymax_C],
            [xmax_C, ymin_C],
            [xmin_C, ymin_C]
        ]]
    }
]

# Reduce size
RR_r = RR.sel(longitude=slice(xmin_C, xmax_C), latitude=slice(ymin_C, ymax_C))
TN_r = TN.sel(longitude=slice(xmin_C, xmax_C), latitude=slice(ymin_C, ymax_C))
TX_r = TX.sel(longitude=slice(xmin_C, xmax_C), latitude=slice(ymin_C, ymax_C))
TG_r = TG.sel(longitude=slice(xmin_C, xmax_C), latitude=slice(ymin_C, ymax_C))
PP_r = PP.sel(longitude=slice(xmin_C, xmax_C), latitude=slice(ymin_C, ymax_C))
HU_r = HU.sel(longitude=slice(xmin_C, xmax_C), latitude=slice(ymin_C, ymax_C))
FG_r = FG.sel(longitude=slice(xmin_C, xmax_C), latitude=slice(ymin_C, ymax_C))
QQ_r = QQ.sel(longitude=slice(xmin_C, xmax_C), latitude=slice(ymin_C, ymax_C))

# albedo 
ALB_r = ALB.sel(longitude=slice(xmin_C, xmax_C), latitude=slice(ymin_C, ymax_C))


# elevation
ELEV_r = ELEV.sel(longitude=slice(xmin_C, xmax_C), latitude=slice(ymin_C, ymax_C))

In [ ]:
# PET last
PET_last_r = PET_last.sel(longitude=slice(xmin_C, xmax_C), latitude=slice(ymin_C, ymax_C))

## 1.2 Reduce temporal dimension 
- this is not needed for elevation as it has no temporal dimension

In [ ]:
# reduce temporal extent
RR_rr = RR_r.sel(time=slice('1981-01-01','2021-12-31'))
TN_rr = TN_r.sel(time=slice('1981-01-01','2021-12-31'))
TX_rr = TX_r.sel(time=slice('1981-01-01','2021-12-31'))
TG_rr = TG_r.sel(time=slice('1981-01-01','2021-12-31'))
PP_rr = PP_r.sel(time=slice('1981-01-01','2021-12-31'))
HU_rr = HU_r.sel(time=slice('1981-01-01','2021-12-31'))
FG_rr = FG_r.sel(time=slice('1981-01-01','2021-12-31'))
QQ_rr = QQ_r.sel(time=slice('1981-01-01','2021-12-31'))

# albedo 
ALB_rr = ALB_r.sel(time=slice('1981-01-01','2021-12-31'))

In [ ]:
# PET last 
PET_last_rr = PET_last_r.sel(time=slice('1981-01-01','2021-12-31'))

In [ ]:
PET_last_rr

## 1.3 Save to NetCDF 

In [ ]:
RR_rr.to_netcdf('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/rr_ens_mean_0.1deg_reg_v26.0e_cl.nc')
TN_rr.to_netcdf('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/tn_ens_mean_0.1deg_reg_v26.0e_cl.nc')
TX_rr.to_netcdf('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/tx_ens_mean_0.1deg_reg_v26.0e_cl.nc')
TG_rr.to_netcdf('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/tg_ens_mean_0.1deg_reg_v26.0e_cl.nc')
PP_rr.to_netcdf('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/pp_ens_mean_0.1deg_reg_v26.0e_cl.nc')
HU_rr.to_netcdf('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/hu_ens_mean_0.1deg_reg_v26.0e_cl.nc')
FG_rr.to_netcdf('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/fg_ens_mean_0.1deg_reg_v26.0e_cl.nc')
QQ_rr.to_netcdf('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/qq_ens_mean_0.1deg_reg_v26.0e_cl.nc')

# albedo
ALB_rr.to_netcdf('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/ALB_ERA5Land_cl.nc')

# elevation 
ELEV_r.to_netcdf('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/elev_ens_0.1deg_reg_v26.0e_cl.nc')

In [ ]:
# PET last 
PET_last_rr.to_netcdf('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/PET_last_ERA5Land_cl.nc')

# 2 Load the clipped files & elevation & shapefile

In [ ]:
# load clipped files
RR_cl_xr = xr.open_dataarray('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/rr_ens_mean_0.1deg_reg_v26.0e_cl.nc', mask_and_scale=True)
TN_cl_xr = xr.open_dataarray('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/tn_ens_mean_0.1deg_reg_v26.0e_cl.nc', mask_and_scale=True)
TX_cl_xr = xr.open_dataarray('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/tx_ens_mean_0.1deg_reg_v26.0e_cl.nc', mask_and_scale=True)
TG_cl_xr = xr.open_dataarray('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/tg_ens_mean_0.1deg_reg_v26.0e_cl.nc', mask_and_scale=True)
PP_cl_xr = xr.open_dataarray('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/pp_ens_mean_0.1deg_reg_v26.0e_cl.nc', mask_and_scale=True)
HU_cl_xr = xr.open_dataarray('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/hu_ens_mean_0.1deg_reg_v26.0e_cl.nc', mask_and_scale=True)
FG_cl_xr = xr.open_dataarray('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/fg_ens_mean_0.1deg_reg_v26.0e_cl.nc', mask_and_scale=True)
QQ_cl_xr = xr.open_dataarray('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/qq_ens_mean_0.1deg_reg_v26.0e_cl.nc', mask_and_scale=True)

# albedo 
ALB_cl_xr = xr.open_dataarray('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/ALB_ERA5Land_cl.nc', mask_and_scale=True)

# elevation
ELEV_cl_xr = xr.open_dataarray('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/elev_ens_0.1deg_reg_v26.0e_cl.nc')

In [ ]:
# PET last
PET_last_cl_xr = xr.open_dataarray('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/PET_last_ERA5Land_cl.nc', mask_and_scale=True)

## 2.1 Write CRS (is this correct?)

In [ ]:
# Define CRS
RR_cl_xr.rio.write_crs("epsg:4326", inplace=True)
TN_cl_xr.rio.write_crs("epsg:4326", inplace=True)
TX_cl_xr.rio.write_crs("epsg:4326", inplace=True)
TG_cl_xr.rio.write_crs("epsg:4326", inplace=True)
PP_cl_xr.rio.write_crs("epsg:4326", inplace=True)
HU_cl_xr.rio.write_crs("epsg:4326", inplace=True)
FG_cl_xr.rio.write_crs("epsg:4326", inplace=True)
QQ_cl_xr.rio.write_crs("epsg:4326", inplace=True)

# albedo 
ALB_cl_xr.rio.write_crs("epsg:4326", inplace=True)

# elevaiton
ELEV_cl_xr.rio.write_crs("epsg:4326", inplace=True)

In [ ]:
# PET last
PET_last_cl_xr.rio.write_crs("epsg:4326", inplace=True)

## 2.2 Load and reproject the shapefile

In [ ]:
# Shapefile 
shape_C = gpd.read_file('/Users/jonathanschieren/Desktop/msc_thesis/data/lamah/hourly/C_basins_intermediate_lowimp/3_shapefiles/Basins_C.shp')

# Reproject to epsg:4326
shape_C_4326 = shape_C.to_crs(epsg=4326)

In [ ]:
print("Before", shape_C.crs)
print("After", shape_C_4326.crs)

In [ ]:
# create array of catchment ID's
IDs = np.zeros(shape_C_4326.shape[0])
for i in range(shape_C_4326.shape[0]):
    IDs[i] = shape_C_4326.ID[i]

## 2.3 Shift and clip the grid of the ERA5Land data (albedo, PET) and shift the grid of wind speed by a tiny bite from 8.05 to 8.0469..
- For some reason the grid of wind speed is 8.05 while all others from EOBS are 8.0469.. it does not really matter but we will fix it here 

In [ ]:
# starting lat 
start_lat_alb = 46.0
start_lon_alb = 8.0

# starting lon 
start_lat_eobs = 46.04986
start_lon_eobs = 8.04986

# differences
lat_shift = start_lat_eobs - start_lat_alb
lon_shift = start_lon_eobs - start_lon_alb

In [ ]:
# shift albedo 
ALB_cl_xr = ALB_cl_xr.assign_coords(latitude=ALB_cl_xr.latitude + lat_shift,
                                              longitude=ALB_cl_xr.longitude + lon_shift)

In [ ]:
# Remove the last value along the latitude dimension
ALB_cl_xr = ALB_cl_xr.isel(latitude=slice(None, -1))

# Remove the last value along the longitude dimension
ALB_cl_xr = ALB_cl_xr.isel(longitude=slice(None, -1))

In [ ]:
# minimally shift grid of wind speed (this is now only affecting the original 10m, as we have alread computed the 2m but does not really matter)
# starting lat 
start_lat_alb = 46.05
start_lon_alb = 8.05

# starting lon 
start_lat_eobs = 46.04986
start_lon_eobs = 8.04986

# differences
lat_shift = start_lat_eobs - start_lat_alb
lon_shift = start_lon_eobs - start_lon_alb


In [ ]:
# shift wind
FG_cl_xr_shifted = FG_cl_xr.assign_coords(latitude=FG_cl_xr.latitude + lat_shift,
                                              longitude=FG_cl_xr.longitude + lon_shift)

In [ ]:
# shift PET last 
PET_last_cl_xr = PET_last_cl_xr.assign_coords(latitude=PET_last_cl_xr.latitude + lat_shift,
                                              longitude=PET_last_cl_xr.longitude + lon_shift)

In [ ]:
# Remove the last value along the latitude dimension
PET_last_cl_xr = PET_last_cl_xr.isel(latitude=slice(None, -1))

# Remove the last value along the longitude dimension
PET_last_cl_xr = PET_last_cl_xr.isel(longitude=slice(None, -1))

In [ ]:
PET_last_cl_xr

# 3. Analyse and fill missing data and scale

## 3.1 Find and interpolate missing values

In [ ]:
# function for missing data 
def missing(dataarray):
    
    # missing values
    missing_values = dataarray.isnull()
    
    # missing values per time step 
    count_missing = missing_values.sum(dim='time')
    
    # total missing 
    total_missing = missing_values.sum()
    
    # percentage missing 
    percentage = total_missing / dataarray.size * 100

    
    return total_missing, percentage

In [ ]:
# mean temperature 
total_missing_TG, percentage_TG = missing(TG_cl_xr)

In [ ]:
# minimum temperature 
total_missing_TN, percentage_TN = missing(TN_cl_xr)

In [ ]:
# maximum temperature
total_missing_TX, percentage_TX = missing(TX_cl_xr)

In [ ]:
# precipitation 
total_missing_RR, percentage_RR = missing(RR_cl_xr)

In [ ]:
# sea level pressure
total_missing_PP, percentage_PP = missing(PP_cl_xr)

In [ ]:
# relative humidity
total_missing_HU, percentage_HU = missing(HU_cl_xr)

In [ ]:
# wind speed 
total_missing_FG, percentage_FG = missing(FG_cl_xr)

In [ ]:
# shortwave radiation 
total_missing_QQ, percentage_QQ = missing(QQ_cl_xr)

In [ ]:
# albedo 
total_missing_ALB, percentage_ALB = missing(ALB_cl_xr)

In [ ]:
# PET last 
total_missing_PET_last, percentage_PET_last = missing(PET_last_cl_xr)

In [ ]:
# print missing values: 
print("Mean temp", total_missing_TG.values, percentage_TG.values)
print("Minimum temp", total_missing_TN.values, percentage_TN.values)
print("Maximum temp", total_missing_TX.values, percentage_TX.values)
print("Precipitation", total_missing_RR.values, percentage_RR.values)
print("Sea level pressure", total_missing_PP.values, percentage_PP.values)
print("Relative humidity", total_missing_HU.values, percentage_HU.values)
print("Wind speed", total_missing_FG.values, percentage_FG.values)
print("Shortwave radiation", total_missing_QQ.values, percentage_QQ.values)
print("Albedo", total_missing_ALB.values, percentage_ALB.values)



In [ ]:
# interpolation function 
def spatial_interpolate_missing(dataarray):
    # Iterate through each time step
    for t in dataarray.time:
        # Check if there are missing values at the current time step
        if dataarray.sel(time=t, method="nearest").isnull().any():
            # Interpolate spatially for the current time step
            dataarray.loc[{"time": t}] = dataarray.sel(time=t, method="nearest").rio.interpolate_na()

    return dataarray

In [ ]:
FG_cl_xr = spatial_interpolate_missing(FG_cl_xr)

In [ ]:
HU_cl_xr = spatial_interpolate_missing(HU_cl_xr)

## 3.2 Convert the PET from ERA5Land from m to mm 
Note: 
**PET in ERA5Land is computed as open water evaporation**

In [ ]:
# convert PET last to mm 
PET_last_mm_cl_xr = PET_last_cl_xr * -1000

In [ ]:
date_test = "1981-04-01"
plot = PET_last_mm_cl_xr.sel(time=date_test)
plot.plot()

# 4. Calculate PET based on Penman-Monteith (FAO) 
- use EOBS observation + albedo from ERA5Land

IMPORTANT: 
**Some functions are taken from pyeto package Github code**
https://github.com/woodcrafty/PyETo/blob/master/pyeto/fao.py

## 4.1 Check and convert units!

TG = daily mean temperature [°C] \
TN = daily minimum temperature [°C] \
TX = daily maximum temperature [°C] \
RR = sum of daily precipitation [mm] \
PP = daily averaged sea level pressure [hPa] \
HU = daily averaged relative humidity [%] \
FG = daily mean wind speed at 10 m height [m/s]\
QQ = surface shortwave downwelling radiation [W/m2] \
ELEV = land surface elevation above sea level [m]

**Which ones do we need for Penman FAO?** \
No changes in unit required: \
- TG = daily mean temperature [°C] \
- TN = daily minimum temperature [°C] \
- TX = daily maximum temperature [°C] \
- FG = daily mean wind speed at 10 m height [m/s] \
- ELEV = land surface elevation above sea level [m]\
- HU = daily averaged relative humidity [%] \

Change in unit required: \
- QQ = surface shortwave downwelling radiation [W/m2] \ -> Needs to be converted to MJ/m2*day

In [ ]:
# surface shortwave downwelling radiation
QQ_cl_xr_mj = (QQ_cl_xr * 86400) * 0.000001

## 4.2 Wind speed [m/s] (correct for altitude)

In [ ]:
# function to correct wind speed
def wind_speed_2m(ws, z):
    """
    Convert wind speed measured at different heights above the soil
    surface to wind speed at 2 m above the surface, assuming a short grass
    surface.

    Based on FAO equation 47 in Allen et al (1998).

    :param ws: Measured wind speed [m s-1]
    :param z: Height of wind measurement above ground surface [m]
    :return: Wind speed at 2 m above the surface [m s-1]
    :rtype: float
    """
    return ws * (4.87 / math.log((67.8 * z) - 5.42))

In [ ]:
# apply conversion 
z = 10 # m 
FG_cl_xr_2m = wind_speed_2m(FG_cl_xr, z)

## 4.3 Slope of saturation vapour pressure curve [kPa / °C]
- Use mean air temperature

In [ ]:
# function
def delta_svp_func(t):
    """
    Estimate the slope of the saturation vapour pressure curve at a given
    temperature.

    Based on equation 13 in Allen et al (1998). If using in the Penman-Monteith
    *t* should be the mean air temperature.

    :param t: Air temperature [deg C]. Use mean air temperature for use in
        Penman-Monteith.
    :return: Saturation vapour pressure [kPa degC-1]
    :rtype: float
    """
    tmp = 4098 * (0.6108 * np.exp((17.27 * t) / (t + 237.3)))
    #return tmp / math.pow((t + 237.3), 2)
    return tmp / np.power((t + 237.3), 2)

In [ ]:
# compoute slope
delta_svp = xr.apply_ufunc(delta_svp_func, TG_cl_xr)

## 4.4 Atmospheric pressure [kPa]
Calculate atmospheric pressure based on elevation data from EOBS 
- This is not a time series!

In [ ]:
# function for atmospheric pressure
def atm_pressure(altitude):
    """
    Estimate atmospheric pressure from altitude.

    Calculated using a simplification of the ideal gas law, assuming 20 degrees
    Celsius for a standard atmosphere. Based on equation 7, page 62 in Allen
    et al (1998).

    :param altitude: Elevation/altitude above sea level [m]
    :return: atmospheric pressure [kPa]
    :rtype: float
    """
    tmp = (293.0 - (0.0065 * altitude)) / 293.0
    return np.power(tmp, 5.26) * 101.3

In [ ]:
# calculate
p_atmos = atm_pressure(ELEV_cl_xr)

## 4.5 Psychrometric constant [kPa / °C]
Calculate psychrometric constant based on atmospheric pressure
- Not a time series

In [ ]:
# function 
def psy_const(atmos_pres):
    """
    Calculate the psychrometric constant.

    This method assumes that the air is saturated with water vapour at the
    minimum daily temperature. This assumption may not hold in arid areas.

    Based on equation 8, page 95 in Allen et al (1998).

    :param atmos_pres: Atmospheric pressure [kPa]. Can be estimated using
        ``atm_pressure()``.
    :return: Psychrometric constant [kPa degC-1].
    :rtype: float
    """
    return 0.000665 * atmos_pres

In [ ]:
psych_const = psy_const(p_atmos)

## 4.6 Mean saturation vapour pressure

In [ ]:
# function for saturation vapour pressure
def svp_from_t(t):
    """
    Estimate saturation vapour pressure (*es*) from air temperature.

    Based on equations 11 and 12 in Allen et al (1998).

    :param t: Temperature [deg C]
    :return: Saturation vapour pressure [kPa]
    :rtype: float
    """
    return 0.6108 * np.exp((17.27 * t) / (t + 237.3))

In [ ]:
# calculate separately saturation vapour pressure for min and max temperature
svp_min = svp_from_t(TN_cl_xr)
svp_max = svp_from_t(TX_cl_xr)

In [ ]:
svp_sum_numpy = svp_max.values + svp_min.values

In [ ]:
coords = {'latitude': svp_min.latitude, 'longitude': svp_min.longitude}

# Convert the NumPy array back into an xarray DataArray
svp_sum = xr.DataArray(svp_sum_numpy, coords=coords, dims=svp_min.dims, attrs=svp_min.attrs)

In [ ]:
svp_mean = svp_sum / 2.0

In [ ]:
# assign time as coordinate, not sure if it makes a difference
svp_mean = svp_mean.assign_coords(time=svp_mean['time'])

In [ ]:
svp_mean

## 4.7 Actual vapour pressure [kPa]
Based on relative humidity. 
- Note that there is also a formula with RHmin and RHmax but we do not have those as observations 

In [ ]:
# function to calculate actual vapour pressure from MEAN relative humidity 
def avp_from_rhmean(svp_tmin, svp_tmax, rh_mean):
    """
    Estimate actual vapour pressure (*ea*) from saturation vapour pressure at
    daily minimum and maximum temperature, and mean relative humidity.

    Based on FAO equation 19 in Allen et al (1998).

    :param svp_tmin: Saturation vapour pressure at daily minimum temperature
        [kPa]. Can be estimated using ``svp_from_t()``.
    :param svp_tmax: Saturation vapour pressure at daily maximum temperature
        [kPa]. Can be estimated using ``svp_from_t()``.
    :param rh_mean: Mean relative humidity [%] (average of RH min and RH max).
    :return: Actual vapour pressure [kPa]
    :rtype: float
    """
    return (rh_mean / 100.0) * ((svp_tmax + svp_tmin) / 2.0)

In [ ]:
# calculate as numpy
avp_numpy = avp_from_rhmean(svp_min.values, svp_max.values, HU_cl_xr.values)

In [ ]:
coords = {'latitude': svp_min.latitude, 'longitude': svp_min.longitude, 'time': svp_min.time}

# Convert the NumPy array back into an xarray DataArray
avp = xr.DataArray(avp_numpy, coords=coords, dims=svp_min.dims, attrs=svp_min.attrs)

## 4.8 Inverse distance & solar declination

### 4.8.1 Inverse distance

In [ ]:
# function for inverse distance 
def inv_rel_dist_earth_sun(day_of_year):
    """
    Calculate the inverse relative distance between earth and sun from
    day of the year.

    Based on FAO equation 23 in Allen et al (1998).

    :param day_of_year: Day of the year [1 to 366]
    :return: Inverse relative distance between earth and the sun
    :rtype: float
    """
    ### _check_doy(day_of_year) # in the original code this is imported to check that DOY is between 0 and 366
    return 1 + (0.033 * np.cos((2.0 * np.pi / 365.0) * day_of_year))

In [ ]:
# first compute day of the year 

# I guess we can take any variable that has a time component so let's take 
# avp 
avp_time = xr.DataArray(pd.to_datetime(avp.time))
doy = avp_time.dt.dayofyear

In [ ]:
# doy
DOY = xr.DataArray(doy, dims='time', coords={'time': avp.time})

In [ ]:
# now we compute the relative dista
inverse_rel_dist = inv_rel_dist_earth_sun(DOY)

### 4.8.2 Solar declination

In [ ]:
# function for solar declination
def sol_dec(day_of_year):
    """
    Calculate solar declination from day of the year.

    Based on FAO equation 24 in Allen et al (1998).

    :param day_of_year: Day of year integer between 1 and 365 or 366).
    :return: solar declination [radians]
    :rtype: float
    """
    ### _check_doy(day_of_year) # in the original code this is imported to check that DOY is between 0 and 366
    return 0.409 * np.sin(((2.0 * np.pi / 365.0) * day_of_year - 1.39))

In [ ]:
solar_declination = sol_dec(DOY)

## 4.9 Convert latitude from degrees to radians [rad]

In [ ]:
# convert (can take any variables)
latitude_rad = avp['latitude'] * np.pi / 180

## 4.10 Sunset hour angle
IMPORTANT: the function is modified to remove the min max part and also the checks are removed

In [ ]:
def sunset_hour_angle(latitude, sol_dec):
    """
    Calculate sunset hour angle (*Ws*) from latitude and solar
    declination.

    Based on FAO equation 25 in Allen et al (1998).

    :param latitude: Latitude [radians]. Note: *latitude* should be negative
        if it in the southern hemisphere, positive if in the northern
        hemisphere.
    :param sol_dec: Solar declination [radians]. Can be calculated using
        ``sol_dec()``.
    :return: Sunset hour angle [radians].
    :rtype: float
    """
    ### _check_latitude_rad(latitude)
    ### _check_sol_dec_rad(sol_dec)

    cos_sha = -np.tan(latitude) * np.tan(sol_dec)
    # If tmp is >= 1 there is no sunset, i.e. 24 hours of daylight
    # If tmp is <= 1 there is no sunrise, i.e. 24 hours of darkness
    # See http://www.itacanet.org/the-sun-as-a-source-of-energy/
    # part-3-calculating-solar-angles/
    # Domain of acos is -1 <= x <= 1 radians (this is not mentioned in FAO-56!)
    
    
    ### return np.arccos(np.min(np.max(cos_sha, -1.0), 1.0)) # removed
    return np.arccos(cos_sha)

In [ ]:
sunset_angle = sunset_hour_angle(latitude_rad, solar_declination)

## 4.11 Extraterrestrial radiation [MJ / m^2 / d]

In [ ]:
# function 
def et_rad(latitude, sol_dec, sha, ird):
    """
    Estimate daily extraterrestrial radiation (*Ra*, 'top of the atmosphere
    radiation').

    Based on equation 21 in Allen et al (1998). If monthly mean radiation is
    required make sure *sol_dec*. *sha* and *irl* have been calculated using
    the day of the year that corresponds to the middle of the month.

    **Note**: From Allen et al (1998): "For the winter months in latitudes
    greater than 55 degrees (N or S), the equations have limited validity.
    Reference should be made to the Smithsonian Tables to assess possible
    deviations."

    :param latitude: Latitude [radians]
    :param sol_dec: Solar declination [radians]. Can be calculated using
        ``sol_dec()``.
    :param sha: Sunset hour angle [radians]. Can be calculated using
        ``sunset_hour_angle()``.
    :param ird: Inverse relative distance earth-sun [dimensionless]. Can be
        calculated using ``inv_rel_dist_earth_sun()``.
    :return: Daily extraterrestrial radiation [MJ m-2 day-1]
    :rtype: float
    """
    ### _check_latitude_rad(latitude)
    ### _check_sol_dec_rad(sol_dec)
    ### _check_sunset_hour_angle_rad(sha)

    tmp1 = (24.0 * 60.0) / np.pi
    tmp2 = sha * np.sin(latitude) * np.sin(sol_dec)
    tmp3 = np.cos(latitude) * np.cos(sol_dec) * np.sin(sha)
    return tmp1 * SOLAR_CONSTANT * ird * (tmp2 + tmp3)

In [ ]:
# calculate 
SOLAR_CONSTANT = 0.0820

extraterrestrial_radiation = et_rad(latitude_rad, solar_declination, sunset_angle, inverse_rel_dist)

## 4.12 Clear-sky radiation [MJ / m2 / d]

In [ ]:
# function 
def cs_rad(altitude, et_rad):
    """
    Estimate clear sky radiation from altitude and extraterrestrial radiation.

    Based on equation 37 in Allen et al (1998) which is recommended when
    calibrated Angstrom values are not available.

    :param altitude: Elevation above sea level [m]
    :param et_rad: Extraterrestrial radiation [MJ m-2 day-1]. Can be
        estimated using ``et_rad()``.
    :return: Clear sky radiation [MJ m-2 day-1]
    :rtype: float
    """
    
    ### added
    altitude_exp = altitude.expand_dims(time=et_rad.time)
    

    return (0.00002 * altitude_exp + 0.75) * et_rad

In [ ]:
# Define the longitude values from 6.05 to 17.95 with a step of 0.1
longitude_values = np.arange(8.04986, 18.04986, 0.1)

# Create a DataArray with the longitude values and latitude coordinate
longitude_da = xr.DataArray(longitude_values, dims=['longitude'])

# Broadcast the extraterrestrial_radiation DataArray along the longitude dimension
extraterrestrial_radiation_gridded = extraterrestrial_radiation.expand_dims(longitude=longitude_values, axis=1)

# Add the longitude values as an attribute
### extraterrestrial_radiation_gridded.attrs['longitude'] = longitude_values

# Now the 'extraterrestrial_radiation_gridded' DataArray has dimensions (latitude=70, longitude=120, time=14976) and the 'longitude_values' as an attribute.

In [ ]:
extraterrestrial_radiation_gridded.rio.write_crs("epsg:4326", inplace=True)

In [ ]:
# compute manually

# Convert DataArrays to NumPy arrays
altitude_np = ELEV_cl_xr.values
et_rad_np = extraterrestrial_radiation_gridded.values

# Get the shapes of the arrays
shape_altitude = altitude_np.shape
shape_et_rad = et_rad_np.shape

# Reshape altitude to match the shape of et_rad (insert a new dimension at index 2)
altitude_np_reshaped = altitude_np[:, :, np.newaxis]

# Transpose et_rad to have the same axis order as altitude
et_rad_np_transposed = np.transpose(et_rad_np, axes=(2, 1, 0))

# Perform element-wise calculation using NumPy arrays
cs_rad_np = (0.00002 * altitude_np_reshaped + 0.75) * et_rad_np_transposed

In [ ]:
clear_sky_radiation = xr.DataArray(cs_rad_np, dims=['latitude', 'longitude', 'time'])

# Copy the coordinates from extraterrestrial_radiation_gridded to cs_rad_dataarray
clear_sky_radiation = clear_sky_radiation.assign_coords(
    latitude=extraterrestrial_radiation_gridded['latitude'],
    longitude=extraterrestrial_radiation_gridded['longitude'],
    time=extraterrestrial_radiation_gridded['time'])

In [ ]:
clear_sky_radiation = clear_sky_radiation.transpose('time', 'latitude', 'longitude')

## 4.13 Net shortwave radiaiton [MJ / m2 / d]
This is calculated based on only observations of shortwave radiation frmo EOBs and albedo fmor ERA5Land

In [ ]:
# function 
def net_in_sol_rad(sol_rad, albedo=0.23):
    """
    Calculate net incoming solar (or shortwave) radiation from gross
    incoming solar radiation, assuming a grass reference crop.

    Net incoming solar radiation is the net shortwave radiation resulting
    from the balance between incoming and reflected solar radiation. The
    output can be converted to equivalent evaporation [mm day-1] using
    ``energy2evap()``.

    Based on FAO equation 38 in Allen et al (1998).

    :param sol_rad: Gross incoming solar radiation [MJ m-2 day-1]. If
        necessary this can be estimated using functions whose name
        begins with 'sol_rad_from'.
    :param albedo: Albedo of the crop as the proportion of gross incoming solar
        radiation that is reflected by the surface. Default value is 0.23,
        which is the value used by the FAO for a short grass reference crop.
        Albedo can be as high as 0.95 for freshly fallen snow and as low as
        0.05 for wet bare soil. A green vegetation over has an albedo of
        about 0.20-0.25 (Allen et al, 1998).
    :return: Net incoming solar (or shortwave) radiation [MJ m-2 day-1].
    :rtype: float
    """
    return (1 - albedo) * sol_rad

In [ ]:
# compute net radiation based on 

#####
"""units"""
#####

net_shortwave_radiation_temp = net_in_sol_rad(QQ_cl_xr_mj.values, ALB_cl_xr.values)

In [ ]:
# re-assign lat lon time 
net_shortwave_radiation = xr.DataArray(net_shortwave_radiation_temp, coords={'latitude': QQ_cl_xr_mj['latitude'], 'longitude': QQ_cl_xr_mj['longitude'], 'time': QQ_cl_xr_mj['time']}, dims=['time', 'latitude', 'longitude'])


## 4.14 Net outgoing longwave radiation [MJ / m2 / d]
Based on: 
- Tmin (EOBS) **Watch out: in Kelvin! So + 273.15** 
- Tmax (EOBS) **Watch out: in Kelvin! So + 273.15** 
- Incoming shortwave radiation (EOBS) 
- Clear-sky radiation (calculated) 
- Actual vapour pressure (calculated) 

In [ ]:
# function 
def net_out_lw_rad(tmin, tmax, sol_rad, cs_rad, avp):
    """
    Estimate net outgoing longwave radiation.

    This is the net longwave energy (net energy flux) leaving the
    earth's surface. It is proportional to the absolute temperature of
    the surface raised to the fourth power according to the Stefan-Boltzmann
    law. However, water vapour, clouds, carbon dioxide and dust are absorbers
    and emitters of longwave radiation. This function corrects the Stefan-
    Boltzmann law for humidity (using actual vapor pressure) and cloudiness
    (using solar radiation and clear sky radiation). The concentrations of all
    other absorbers are assumed to be constant.

    The output can be converted to equivalent evaporation [mm day-1] using
    ``energy2evap()``.

    Based on FAO equation 39 in Allen et al (1998).

    :param tmin: Absolute daily minimum temperature [degrees Kelvin]
    :param tmax: Absolute daily maximum temperature [degrees Kelvin]
    :param sol_rad: Solar radiation [MJ m-2 day-1]. If necessary this can be
        estimated using ``sol+rad()``.
    :param cs_rad: Clear sky radiation [MJ m-2 day-1]. Can be estimated using
        ``cs_rad()``.
    :param avp: Actual vapour pressure [kPa]. Can be estimated using functions
        with names beginning with 'avp_from'.
    :return: Net outgoing longwave radiation [MJ m-2 day-1]
    :rtype: float
    """
    
    
    ############## add 273.15 here for Kelvin, correct??
    tmp1 = (STEFAN_BOLTZMANN_CONSTANT *
        ((np.power((tmax+273.15), 4) + np.power((tmin+273.15), 4)) / 2))
    tmp2 = (0.34 - (0.14 * np.sqrt(avp)))
    tmp3 = 1.35 * (sol_rad / cs_rad) - 0.35
    return tmp1 * tmp2 * tmp3

In [ ]:
# Stefan Boltzmann constant [MJ K-4 m-2 day-1]
STEFAN_BOLTZMANN_CONSTANT = 0.000000004903  #
"""Stefan Boltzmann constant [MJ K-4 m-2 day-1]"""

In [ ]:
# calculate 
####
"""units"""
"""Makse sure not only QQ is the right units, also the equation above has been adjusted to correct the °C input to Kelvin"""
####


net_out_longwave_np = net_out_lw_rad(TN_cl_xr.values, TX_cl_xr, QQ_cl_xr_mj.values, clear_sky_radiation.values, avp.values)



In [ ]:
# re-assign lat lon time 
net_out_longwave = xr.DataArray(net_out_longwave_np, coords={'latitude': QQ_cl_xr_mj['latitude'], 'longitude': QQ_cl_xr_mj['longitude'], 'time': QQ_cl_xr_mj['time']}, dims=['time', 'latitude', 'longitude'])


## 4.15 Net Radiation [MJ / m2 / d]
Rn = Rns_in - Rnl_out

In [ ]:
net_radiation = net_shortwave_radiation - net_out_longwave

In [ ]:
net_radiation

## 4.16 Reference Evapotranspiration
- **Pay attention and double-check the units**
- Either correct for Kelvin or use Kelvin as input

In [ ]:
### modified with temperature correction for Kelcin
def fao56_penman_monteith(net_rad, t, ws, svp, avp, delta_svp, psy, shf=0.0):
    """
    Estimate reference evapotranspiration (ETo) from a hypothetical
    short grass reference surface using the FAO-56 Penman-Monteith equation.

    Based on equation 6 in Allen et al (1998).

    :param net_rad: Net radiation at crop surface [MJ m-2 day-1]. If
        necessary this can be estimated using ``net_rad()``.
    :param t: Air temperature at 2 m height [deg Kelvin].
    :param ws: Wind speed at 2 m height [m s-1]. If not measured at 2m,
        convert using ``wind_speed_at_2m()``.
    :param svp: Saturation vapour pressure [kPa]. Can be estimated using
        ``svp_from_t()''.
    :param avp: Actual vapour pressure [kPa]. Can be estimated using a range
        of functions with names beginning with 'avp_from'.
    :param delta_svp: Slope of saturation vapour pressure curve [kPa degC-1].
        Can be estimated using ``delta_svp()``.
    :param psy: Psychrometric constant [kPa deg C]. Can be estimatred using
        ``psy_const_of_psychrometer()`` or ``psy_const()``.
    :param shf: Soil heat flux (G) [MJ m-2 day-1] (default is 0.0, which is
        reasonable for a daily or 10-day time steps). For monthly time steps
        *shf* can be estimated using ``monthly_soil_heat_flux()`` or
        ``monthly_soil_heat_flux2()``.
    :return: Reference evapotranspiration (ETo) from a hypothetical
        grass reference surface [mm day-1].
    :rtype: float
    """
    a1 = (0.408 * (net_rad - shf) * delta_svp /
          (delta_svp + (psy * (1 + 0.34 * ws))))
    a2 = (900 * ws / (t + 273.15) * (svp - avp) * psy /
          (delta_svp + (psy * (1 + 0.34 * ws))))
    return a1 + a2

In [ ]:
# compute
ET0 = fao56_penman_monteith(net_radiation.values, TG_cl_xr.values, FG_cl_xr_2m.values, svp_mean.values, avp.values, delta_svp.values, psych_const.values)


In [ ]:
ET0

In [ ]:
# save dataarray as netcdf file 
ET0_fao = xr.DataArray(ET0, coords={'latitude': TG_cl_xr['latitude'], 'longitude': TG_cl_xr['longitude'], 'time': TG_cl_xr['time']}, dims=['time', 'latitude', 'longitude'])

In [ ]:
date_test = "1981-07-01"
plot = ET0_fao.sel(time=date_test)
plot.plot()

In [ ]:
# Step 5: Save the result to a new netCDF file
ET0_fao.to_netcdf('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/ET0_penman_monteith_fao_itp.nc')

In [ ]:
# plot it 
date_test = "1981-01-01"
plot = ET0_fao.sel(time=date_test)
plot.plot()

Final variables: 
- temperature not changed [°C]
- wind: FG:cl_xr_2m [m/s]
- slope of saturation vapour pressure curve: delta_svp [kPa / °C]
- Atmospheric pressure: p_atmos[kPa] - not a time series
- psychrometric constant: psych_const [kPa / °C] - not a time series
- mean saturation vapour pressure: svp_mean [kPa]
- inverse relative disctance: inverse_rel_dist [m ??]
- solar declination: solar_declination
- sunset_angle
- extraterrestrial_radiation [MJ / m2 / d]
- clear_sky_radiation
- net_shortwave_radiation
- net_out_longwave
- net_radiation

# 5. Zonal Stats with Area Weightings (uxing Xagg)
https://climateestimate.net/content/example-step3.html

In [ ]:
# create weightmap, can be any variable I think
weightmap = xa.pixel_overlaps(RR_cl_xr, shape_C_4326,subset_bbox=False)


In [ ]:
# weightmap PET last (it's the same just repeating) 
weightmap = xa.pixel_overlaps(PET_last_mm_cl_xr, shape_C_4326,subset_bbox=False)

In [ ]:
# weightmap PET last (it's the same just repeating) 
weightmap = xa.pixel_overlaps(ET0_fao, shape_C_4326,subset_bbox=False)

## 5.1 Aggregate EOBS vars, Albedo and PET to catchments

In [ ]:
# minimum temperature
TN_aggregated_xagg = xa.aggregate(TN_cl_xr, weightmap)

In [ ]:
# save to csv 
TN_aggregated_xagg.to_csv('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_new/TN_aggregated_xagg_xr.csv')

In [ ]:
# mean temperature
TG_aggregated_xagg = xa.aggregate(TG_cl_xr, weightmap)

In [ ]:
# save to csv
TG_aggregated_xagg.to_csv('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_new/TG_aggregated_xagg_xr.csv')

In [ ]:
# maximum temperature
TX_aggregated_xagg = xa.aggregate(TX_cl_xr, weightmap)

In [ ]:
# save to csv
TX_aggregated_xagg.to_csv('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_new/TX_aggregated_xagg_xr.csv')

In [ ]:
# precipitation 
RR_aggregated_xagg = xa.aggregate(RR_cl_xr, weightmap)

In [ ]:
# save to csv 
RR_aggregated_xagg.to_csv('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_new/RR_aggregated_xagg_xr.csv')


In [ ]:
# sea level pressure
PP_aggregated_xagg = xa.aggregate(PP_cl_xr, weightmap)

In [ ]:
# save to csv 
PP_aggregated_xagg.to_csv('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_new/PP_aggregated_xagg_xr.csv')


In [ ]:
# relative humidity 
HU_aggregated_xagg = xa.aggregate(HU_cl_xr, weightmap)

In [ ]:
# save to csv 
HU_aggregated_xagg.to_csv('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_new/HU_aggregated_xagg_xr.csv')


In [ ]:
# wind speed # make sure to take the right one! 
# we are now taking the original at 10 m, not the one we used in the Penman equation
FG_aggregated_xagg = xa.aggregate(FG_cl_xr_shifted, weightmap)

In [ ]:
# save to csv 
FG_aggregated_xagg.to_csv('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_new/FG_aggregated_xagg_xr.csv')


In [ ]:
# shortwave radiation 
QQ_aggregated_xagg = xa.aggregate(QQ_cl_xr, weightmap)

In [ ]:
# save to csv 
QQ_aggregated_xagg.to_csv('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_new/QQ_aggregated_xagg_xr.csv')


In [ ]:
# albedo 
ALB_aggregated_xagg = xa.aggregate(ALB_cl_xr, weightmap)

In [ ]:
# save to csv 
ALB_aggregated_xagg.to_csv('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_new/ALB_aggregated_xagg_xr.csv')

In [ ]:
# calculated PET fao 
# albedo 
ET0_fao_aggregated_xagg = xa.aggregate(ET0_fao, weightmap)

In [ ]:
# save to csv 
ET0_fao_aggregated_xagg.to_csv('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_new/ET0_fao_aggregated_xagg_xr.csv')


In [ ]:
# PET from ERA5Land last
PET_last_aggregated_xagg = xa.aggregate(PET_last_mm_cl_xr, weightmap)

In [ ]:
# save to CSV
PET_last_aggregated_xagg.to_csv('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_new/PET_last_era5land_aggregated_xagg_xr.csv')


## 5.2 Save as individual catchments

In [ ]:
# load minimum temp
TN_xagg = pd.read_csv('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_new/TN_aggregated_xagg_xr.csv', usecols=['ID', 'time', 'tn'])

In [ ]:
# load mean temp
TG_xagg = pd.read_csv('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_new/TG_aggregated_xagg_xr.csv', usecols=['ID', 'time', 'tg'])

In [ ]:
# maximum temp
TX_xagg = pd.read_csv('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_new/TX_aggregated_xagg_xr.csv', usecols=['ID', 'time', 'tx'])

In [ ]:
# load precipitation
RR_xagg = pd.read_csv('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_new/RR_aggregated_xagg_xr.csv', usecols=['ID', 'time', 'rr'])

In [ ]:
# load sea level pressure
PP_xagg = pd.read_csv('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_new/PP_aggregated_xagg_xr.csv', usecols=['ID', 'time', 'pp'])

In [ ]:
# load relative humidity
HU_xagg = pd.read_csv('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_new/HU_aggregated_xagg_xr.csv', usecols=['ID', 'time', 'hu'])

In [ ]:
# wind speed 
FG_xagg = pd.read_csv('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_new/FG_aggregated_xagg_xr.csv', usecols=['ID', 'time', 'fg'])

In [ ]:
# shortwave radiation 
QQ_xagg = pd.read_csv('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_new/QQ_aggregated_xagg_xr.csv', usecols=['ID', 'time', 'qq'])

In [ ]:
# open albedo 
ALB_xagg = pd.read_csv('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_new/ALB_aggregated_xagg_xr.csv', usecols=['ID', 'time', 'fal'])

In [ ]:
# open ET0 Penman 
ET0_xagg = pd.read_csv('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_new/ET0_fao_aggregated_xagg_xr.csv', usecols=['ID', 'time', 'var'])


In [ ]:
# rename column in ET0 Penman 
ET0_xagg.rename(columns={'var': 'ET0'}, inplace=True)

In [ ]:
ET0_xagg # pet_2.set_index('time', inplace=True)

In [ ]:
# open PET last 
PET_last_xagg = pd.read_csv('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_new/PET_last_era5land_aggregated_xagg_xr.csv', usecols=['ID', 'time', 'pev'])


In [ ]:
PET_last_xagg

In [ ]:
# save all catchments as individual csv files 
path = '/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_individual_et0/'
for IDs in shape_C_4326.ID:
    
    print(IDs)
    et0 = ET0_xagg[ET0_xagg['ID']==IDs]

    
    # drop the ID columns drop time columns for all vars except precipitation
    et0 = et0.drop(['ID'], axis = 1)

    et0.to_csv(path + "ID_" + str(IDs) + ".csv")


In [ ]:
def invert_pet(input_folder1, input_folder2, output_folder, id_list):
    for id_num in id_list:
        input_file1 = os.path.join(input_folder1, f"ID_{id_num}.csv")
        input_file2 = os.path.join(input_folder2, f"ID_{id_num}.csv")
        output_file = os.path.join(output_folder, f"ID_{id_num}.csv")
        
        # read
        data1 = pd.read_csv(input_file1)
        data2 = pd.read_csv(input_file2)
        
        selected_columns = ['time', 'rr', 'tg', 'tn', 'tx', 'pp', 'hu', 'fg', 'qq', 'fal', 'DOY', 'daylength', 'pev']
        combined_data = pd.concat([data1[selected_columns], data2[['ET0']]], axis=1)
        combined_data.set_index('time', inplace=True)
        
        # Save the updated data to a new CSV file
        combined_data.to_csv(output_file, index=True)
        
        print(f"ID: {id_num}")

In [ ]:
IDs = IDs.astype(int)

In [ ]:
IDs

In [ ]:
# Specify input and output paths and ID list
input_folder_path1 = "/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_individual_final/timeseries/forcings_c_xagg/"
input_folder_path2 = "/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_individual_et0/"
output_folder_path = "/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_timeseries/"

# Call the function
invert_pet(input_folder_path1, input_folder_path2, output_folder_path, IDs)

In [ ]:
# final 
final = pd.read_csv('/Users/jonathanschieren/Desktop/msc_thesis/data/e_obs/xagg_timeseries/ID_196.csv')


In [ ]:
final

# 6. Runoff Time Series (LamaH-CE)
Runoff time series in LamaH were interpolated when gaps where smaller than 6 h. 

Here we analyze how long the time series are and which ones contain gaps and how large those gaps are. 



## 6.1 Gauge Attributes

In [ ]:
# IDs in catchment delineation C (this is created further up) 
IDs

In [ ]:
# load file with gauge attributes (this is for ALL gauges)
gauge_attrs = pd.read_csv("/Users/jonathanschieren/Desktop/msc_thesis/data/lamah/hourly/D_gauges/1_attributes/Gauge_attributes_MOD.csv", delimiter=",")


In [ ]:
gauge_attrs.head(2)

In [ ]:
# subset of gauge attributes that includes only the gauges used in catchment delineation C
gauge_attrs_C = gauge_attrs[gauge_attrs['ID'].isin(IDs)]
gauge_attrs_C.head()

In [ ]:
gauge_attrs_C.columns

In [ ]:
gap_count = gauge_attrs_C['gaps_post'].value_counts()

In [ ]:
gap_count

In [ ]:
plt.bar(gauge_attrs_C['ID'], gauge_attrs_C['gaps_post'])

In [ ]:
# df that includes only the gauges/IDs that are gap free
gauge_attrs_C_nogap = gauge_attrs_C[gauge_attrs_C['gaps_post'] == 0.0]

# array with gap free IDs
IDs_nogap = gauge_attrs_C[gauge_attrs_C['gaps_post'] == 0.0]['ID'].values

In [ ]:
gauge_attrs_C_nogap.tail()

In [ ]:
gauge_attrs_C_nogap.shape

## 6.2 Inspect start and end time of the gap-free time series

In [ ]:
runoff_path = "/Users/jonathanschieren/Desktop/msc_thesis/data/lamah/hourly/D_gauges/2_timeseries/daily/"


In [ ]:
csv_files = glob.glob(os.path.join(runoff_path, "ID_*.csv"))

for file_path in csv_files:
    file_name = os.path.basename(file_path)
    file_id = int(file_name.split('_')[1][:-4])  # Extract the ID from the file name
    
    if file_id in IDs_nogap:
        df = pd.read_csv(file_path, delimiter=";")

        first_date = df.iloc[0]
        last_date = df.iloc[-1]
        print(f"File: {file_name}")
        first = first_date['YYYY'].astype(int)
        last = last_date['YYYY'].astype(int)
        print(f"First Date: {first_date['YYYY'].astype(int)}-{first_date['MM'].astype(int)}-{first_date['DD'].astype(int)}")
        print(f"Last Date: {last_date['YYYY'].astype(int)}-{last_date['MM'].astype(int)}-{last_date['DD'].astype(int)}")
        print("")


In [ ]:
# Date range boundaries
start_date = dt(1981, 1, 1)
end_date = dt(2017, 12, 31)

# List to store the filtered IDs
filtered_ids = []

# Loop through the CSV files in the folder
csv_files = glob.glob(os.path.join(runoff_path, "ID_*.csv"))

for file_path in csv_files:
    file_name = os.path.basename(file_path)
    file_id = int(file_name.split('_')[1][:-4])  # Extract the ID from the file name
    
    if file_id in IDs_nogap:
        df = pd.read_csv(file_path, delimiter=";")
        
        # Convert "YYYY", "MM", "DD" columns to a datetime object
        df['Date'] = pd.to_datetime(df[['YYYY', 'MM', 'DD']].astype(str).apply('-'.join, axis=1))
        
        first_date = df.iloc[0]['Date']
        last_date = df.iloc[-1]['Date']
        
        if start_date == first_date and last_date == end_date:
            filtered_ids.append(file_id)
            
            print(f"File: {file_name}")
            print(f"First Date: {first_date}")
            print(f"Last Date: {last_date}")
            print("")

print("Filtered IDs:", filtered_ids)

In [ ]:
# save filtered IDs 
csv_file_path = "/Users/jonathanschieren/Desktop/msc_thesis/data/lamah/hourly/C_basins_intermediate_lowimp/IDs_nogap_fullperiod.csv"
pd.DataFrame({"Filtered IDs": filtered_ids}).to_csv(csv_file_path, index=False)